In [171]:

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)



In [172]:
import pandas as pd

test= pd.read_csv(r"C:\Users\alesi\Documents\group_project_ml\test.csv")
train= pd.read_csv(r"C:\Users\alesi\Documents\group_project_ml\train.csv")

In [173]:
financial_train = train.drop("y", axis=1) # drop labels for training 
y=train["y"].copy()
financial_train2 = financial_train.drop("w", axis=1)


In [174]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))

fin_array = scaler.fit_transform(financial_train2)


In [175]:
features_set = np.reshape(fin_array, ( fin_array.shape[0],fin_array.shape[1],1))

In [176]:
features_set


array([[[0.77543941],
        [0.73641688],
        [0.66819053],
        ...,
        [0.81500202],
        [0.80270631],
        [0.70407194]],

       [[0.76573625],
        [0.69986236],
        [0.62592815],
        ...,
        [0.85070807],
        [0.82899471],
        [0.84460308]],

       [[0.7421511 ],
        [0.7122393 ],
        [0.76833876],
        ...,
        [0.61630346],
        [0.62767519],
        [0.68464607]],

       ...,

       [[0.84564578],
        [0.83921647],
        [0.83300523],
        ...,
        [0.78301731],
        [0.77935849],
        [0.78419072]],

       [[0.79846584],
        [0.79272539],
        [0.79541808],
        ...,
        [0.77660452],
        [0.77457619],
        [0.78108037]],

       [[0.91306063],
        [0.90600368],
        [0.89643146],
        ...,
        [0.77566848],
        [0.7717734 ],
        [0.77295746]]])

In [177]:
features_set.shape

(7326, 50, 1)

In [178]:
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed, RepeatVector
from keras.layers import LSTM, GRU
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

In [179]:
early_stopping = EarlyStopping(monitor="val_loss",
    min_delta=0.004, 
    patience=10, 
    restore_best_weights=True,
)

In [180]:
labels=np.array(y)
labels
labels.shape


(7326,)

In [181]:
model = Sequential()

model.add(LSTM(units=50, return_sequences="True" ,input_shape=(features_set.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences="True" ,input_shape=(features_set.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, input_shape=(features_set.shape[1], 1)))
model.add(Dropout(0.2))

model.add(Dense(units = 1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(features_set, labels, epochs = 20,validation_split=0.2, batch_size = 32)

Epoch 1/20
184/184 [==============================] - 16s 42ms/step - loss: 0.3359 - val_loss: 0.1589
Epoch 2/20
184/184 [==============================] - 7s 37ms/step - loss: 0.1256 - val_loss: 0.1380
Epoch 3/20
184/184 [==============================] - 7s 38ms/step - loss: 0.1102 - val_loss: 0.1129
Epoch 4/20
184/184 [==============================] - 7s 38ms/step - loss: 0.1026 - val_loss: 0.1393
Epoch 5/20
184/184 [==============================] - 7s 39ms/step - loss: 0.0812 - val_loss: 0.1116
Epoch 6/20
179/184 [============================>.] - ETA: 0s - loss: 0.0829

KeyboardInterrupt: 

In [182]:

model.fit(features_set, labels, epochs = 100,validation_split=0.1, batch_size = 28, callbacks=[early_stopping])


Epoch 1/100
236/236 [==============================] - 8s 36ms/step - loss: 0.0788 - val_loss: 0.0812
Epoch 2/100
 27/236 [==>...........................] - ETA: 6s - loss: 0.0697

KeyboardInterrupt: 

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def build_classifier(optimizer):
    grid_model = Sequential()
    grid_model.add(LSTM(units = 64,input_shape = (50,1)))
    grid_model.add(Dropout(0.4))
    grid_model.add(Dense(1))

    grid_model.compile(loss = 'mse',optimizer = optimizer, metrics = ['mean_squared_error'])
    return grid_model

grid_model = KerasClassifier(build_fn=build_classifier)
parameters = {'batch_size' : [32],
               'epochs' : [20,50],
              'optimizer' : ['adam'] }

grid_search  = GridSearchCV(estimator = grid_model,
                            param_grid = parameters,
                            scoring = 'neg_mean_squared_error',
                            cv = 2)



grid_search = grid_search.fit(features_set,labels)

In [ ]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
best_parameters

In [ ]:

test_scaled=scaler.transform(test)
test_scaled


In [39]:

test_features = np.reshape(test_scaled, (-1, test_scaled.shape[1], 1))
test_features.shape


test_features.shape

(3141, 50, 1)

In [159]:
predictions = model.predict(test_features)
predictions

array([[-0.43264723],
       [ 0.28264827],
       [ 1.5591255 ],
       ...,
       [ 1.0240332 ],
       [ 1.0181649 ],
       [ 1.0232127 ]], dtype=float32)

In [163]:
np.savetxt("submissions23.txt", predictions,delimiter = ",")